# Notebook Instructions

1. If you are new to Jupyter notebooks, please go through this introductory manual <a href='https://quantra.quantinsti.com/quantra-notebook' target="_blank">here</a>.
1. Any changes made in this notebook would be lost after you close the browser window. **You can download the notebook to save your work on your PC.**
1. Before running this notebook on your local PC:<br>
i.  You need to set up a Python environment and the relevant packages on your local PC. To do so, go through the section on "**Run Codes Locally on Your Machine**" in the course.<br>
ii. You need to **download the zip file available in the last unit** of this course. The zip file contains the data files and/or python modules that might be required to run this notebook.

# Backtest Short Straddle Strategy

In the previous section, we forecasted the implied volatility values using a Random Forest regressor. In this notebook, we will backtest a short straddle strategy that takes trades based on the predicted IV values.

The notebook is structured as follows:
1. [Import the Data](#import)
2. [Strategy Parameters and Risk Management](#parameters)
3. [Entry Condition](#entry)
4. [Exit Condition](#exit)
5. [Backtesting and Performance Analysis](#backtesting)

In [1]:
# For data manipulation
import numpy as np
import pandas as pd

# Helper functions
import sys
sys.path.append("..")
from data_modules.ml_options_util_quantra import trade_level_analytics

<a id='import'></a>
## Import the Data

Import the file `data_pred.csv` using the `read_csv` method of `pandas`. This file contains the predicted IV values from the previous notebook.
This CSV file is available in the zip file of the unit 'Python Codes and Data' in the 'Course Summary' section.

In [2]:
# Read the data
options_data = pd.read_csv("../data_modules/data_pred_forecasting_iv.csv", index_col=0)

# Change the index type to datetime
options_data.index = pd.to_datetime(options_data.index)

# Display the data
options_data.tail()

,STRIKE,STRIKE_DISTANCE_PCT,DTE,C_IV,P_IV,C_LAST,P_LAST,UNDERLYING_LAST,ATM,Open,...,Close,Volume,RSI,NATR,ADX,NORM_MIDDLE,NEXT_DAY_PUT_IV,NEXT_DAY_CALL_IV,predicted_put_IV,predicted_call_IV
[QUOTE_DATE],,,,,,,,,,,,,,,,,,,,,
2022-09-22,3755.0,0.001,8.0,0.26337,0.25325,70.0,53.90,3756.86,3755.0,3782.360107,...,3757.989990,4.284600e+09,32.057740,1.950751,21.700846,1.020602,0.28858,0.27027,0.220681,0.224350
2022-09-23,3695.0,0.000,7.0,0.27027,0.28858,57.8,66.68,3695.49,3695.0,3727.139893,...,3693.229980,5.144270e+09,28.904365,2.056925,23.335447,1.028746,0.34657,0.34644,0.236011,0.270536
2022-09-26,3655.0,0.000,4.0,0.34644,0.34657,64.9,52.00,3656.13,3655.0,3682.719971,...,3655.040039,4.886140e+09,27.204884,2.068534,24.871937,1.026097,0.33897,0.32786,0.286969,0.284785
2022-09-27,3650.0,0.000,3.0,0.32786,0.33897,48.0,47.66,3648.49,3650.0,3686.439941,...,3647.290039,4.577740e+09,26.859747,2.109422,26.450913,1.016836,0.27130,0.31591,0.276878,0.310770
2022-09-28,3720.0,0.001,2.0,0.31591,0.27130,40.1,31.90,3717.69,3720.0,3651.939941,...,3719.040039,4.684850e+09,35.072333,2.105589,27.540235,0.993406,0.30141,0.30883,0.237296,0.339038


<a id='parameters'></a>
## Strategy Parameters and Risk Management

We will set the stop-loss (SL) and take-profit (TP) percentage to 30% and 60% of the net entry premium, respectively. You can try changing this to see how it affects the backtest results. However, if the SL is kept too low, it might get hit too frequently. On the other hand, if we keep it too high, it might not hit at all. We will keep the take-profit at 60% as it provides a good risk-to-reward ratio of 1:2.

The `days_to_exit_before_expiry` can be changed if you want to exit the trade a few days before expiry and don't want to hold it till expiry to avoid a huge mark-to-market (MTM) swing when expiry is near.

In [3]:
config = {
    'stop_loss_percentage': 30,
    'take_profit_percentage': 60,
    'days_to_exit_before_expiry': 0
}

<a id='entry'></a>
## Entry Condition

We will check the following condition for entry:
1. The mean of `predicted_call_IV` and `predicted_put_IV` should be greater than `0.2`.
2. The mean of the `predicted_call_IV` and `predicted_put_IV` should be greater than the mean of the `call_iv` and `put_iv`. Thus, the predicted mean IV should be less than the current mean IV.

Create a column `signal_iv` which stores `1` when the above conditions are met.

**Note:** We have taken the threshold IV value as `0.2` here based on the observation of the underlying asset's volatility. You can try changing the threshold value as per your asset's volatility.

In [4]:
# Calculate mean IVs
options_data["current_mean"] = (options_data['C_IV'] + options_data['P_IV'])/2
options_data["pred_mean"] = (
    options_data['predicted_call_IV'] + options_data['predicted_put_IV'])/2

# IV entry condition
condition_1 = (options_data["current_mean"] > 0.2)
condition_2 = (options_data["pred_mean"] < options_data["current_mean"])

# Generate signal as 1 when the condition is true
options_data['signal_iv'] = np.where(condition_1 & condition_2, 1, np.nan)

<a id='exit'></a>
## Exit Condition

We will exit the trade if the net_premium on the given date either crosses above the take-profit, or below the stop-loss, or at expiry. The conditions for SL and TP have been defined in the next part of the code.

`signal_iv` column stores `0` when `days_to_expiry` is less than `days_to_exit_before_expiry`.

In [5]:
# Generate signal as 0 when days to expiry is less than days to exit before expiry
options_data['signal_iv'] = np.where(
    options_data.DTE <= config['days_to_exit_before_expiry'], 0, options_data['signal_iv'])

# Display the data
options_data.head()

,STRIKE,STRIKE_DISTANCE_PCT,DTE,C_IV,P_IV,C_LAST,P_LAST,UNDERLYING_LAST,ATM,Open,...,NATR,ADX,NORM_MIDDLE,NEXT_DAY_PUT_IV,NEXT_DAY_CALL_IV,predicted_put_IV,predicted_call_IV,current_mean,pred_mean,signal_iv
[QUOTE_DATE],,,,,,,,,,,,,,,,,,,,,
2020-06-12,3040.0,0.000,18.0,0.31362,0.28960,77.80,113.55,3040.49,3040.0,3071.040039,...,2.250210,21.002960,1.030682,0.29755,0.28138,0.243739,0.281487,0.301610,0.262613,1.0
2020-06-15,3070.0,0.001,15.0,0.28138,0.29755,75.91,81.30,3067.68,3070.0,2993.760010,...,2.338024,20.341381,1.011372,0.30209,0.28269,0.247606,0.277366,0.289465,0.262486,1.0
2020-06-16,3125.0,0.000,14.0,0.28269,0.30209,68.92,77.50,3126.22,3125.0,3131.000000,...,2.329174,19.095049,0.987274,0.27824,0.28178,0.253833,0.291827,0.292390,0.272830,1.0
2020-06-17,3115.0,0.001,13.0,0.28178,0.27824,67.98,72.63,3113.06,3115.0,3136.129883,...,2.246624,17.937740,0.985918,0.19829,0.32941,0.240493,0.300708,0.280010,0.270600,1.0
2020-06-19,3095.0,0.001,11.0,0.32941,0.19829,64.40,65.50,3097.28,3095.0,3140.290039,...,2.170696,15.974768,1.001885,0.25004,0.24480,0.214533,0.310770,0.263850,0.262652,1.0


<a id='backtesting'></a>
## Backtesting and Performance Analysis

We will loop over each of the dates in the data, set up the straddle when entry conditions are met, exit when exit conditions are met, and update the trade in `round_trips_details`. 

`mark_to_market` dataframe contains the premiums of the strategy on each date between the entry date and exit date.

We will backtest the straddle using the following steps:

**Step-1**: Create dataframes `round_trips_details`, `trades` and `mark_to_market` for storing round trips, trades, and mtm, respectively.

**Step-2**: Define a function `add_to_mtm` which stores daily mark_to_market values for the strategy. It takes the existing `mark_to_market` dataframe, `option_strategy` which is a straddle in this case and `trading_date` as inputs.

**Step-3**: Define a function `get_premium` to get the premium for the two legs(CE and PE) of the straddle. It takes `options_straddle` and `options_data` as inputs.

**Step-4**: Define a function `setup_straddle` to set up the straddle strategy. It takes `options_data` and `direction` (long/short) as inputs.

**Step-5**: Initialise `current_position`, `trade_num`, which is basically the number of trades, `cum_pnl` to 0 and set the `entry_flag` to `False`. 

**Step-6**: We also set the `start_date` for backtesting.

In [6]:
# Create dataframes for round trips, storing trades, and mtm
round_trips_details = pd.DataFrame()
trades = pd.DataFrame()
mark_to_market = pd.DataFrame()

# Function for calculating mtm


def add_to_mtm(mark_to_market, option_strategy, trading_date):
    option_strategy['Date'] = trading_date
    mark_to_market = pd.concat([mark_to_market, option_strategy])
    return mark_to_market

# Function for fetching premium


def get_premium(options_strategy, options_data):

    # Get the premium for call option
    if options_strategy['Option Type'] == "CE":
        return options_data['C_LAST']

    # Get the premium for put option
    elif options_strategy['Option Type'] == "PE":
        return options_data['P_LAST']

# Function for setting up a straddle


def setup_straddle(options_data, direction='short'):

    # Create a dataframe to store the straddle
    straddle = pd.DataFrame()

    # CE and PE legs of the straddle
    straddle['Option Type'] = ['CE', 'PE']

    # Create the straddle at ATM
    straddle['Strike Price'] = options_data.ATM

    # Sell positions for both CE and PE legs in case of a short straddle
    straddle['position'] = -1

    # Get the premiums for the two option legs of the short straddle
    straddle['premium'] = straddle.apply(
        lambda r: get_premium(r, options_data), axis=1)

    # Multiply the position by -1 in case of a long straddle
    if direction == 'long':
        straddle['position'] *= -1

    # Get the premiums for the two option legs of the long straddle
    straddle['premium'] = straddle.apply(
        lambda r: get_premium(r, options_data), axis=1)

    return straddle


# Initialise current position, number of trades and cumulative pnl to 0
current_position = 0
trade_num = 0
cum_pnl = 0

# Set exit flag to False
exit_flag = False

# Set start date for backtesting
start_date = options_data.index[0]

Perform the following steps iteratively for the dates in the backtest period.

**Step-7**: For a given date, if there is no open position and entry conditions are met, we will set up the straddle.

**Step-8**: For a given date, if there is an open position, we exit the trade if the stop-loss/take-profit gets hit or if the given date is an expiry and update round trips.

**Step-9**: Finally, we calculate the pnl for each trade and also the cumulative pnl.

In [7]:
for i in options_data.loc[start_date:].index:

    if (current_position == 0) and options_data.loc[i, 'signal_iv'] == 1:

        # Setup straddle
        straddle = setup_straddle(options_data.loc[i], direction="short")

        # Populate the trades dataframe
        trades = straddle.copy()
        trades['entry_date'] = i
        trades.rename(columns={'premium': 'entry_price'}, inplace=True)

        # Calculate net premium
        net_premium = round((straddle.position * straddle.premium).sum(), 1)

        # Compute SL and TP for the trade
        premium_sign = np.sign(net_premium)
        sl = net_premium * \
            (1 - config['stop_loss_percentage']*premium_sign/100)
        tp = net_premium * \
            (1 + config['take_profit_percentage']*premium_sign/100)

        # Update current position to 1
        current_position = 1

        # Update mark_to_market dataframe
        mark_to_market = add_to_mtm(mark_to_market, straddle, i)

        # Increase number of trades by 1
        trade_num += 1
        print("-"*30)

        # Print trade details
        print(
            f"Trade No: {trade_num} | Entry | Date: {i} | Premium: {net_premium} | Pnl: 0 | Cum PnL: {cum_pnl}")

    elif current_position == 1:

        # Update net premium
        straddle['premium'] = straddle.apply(
            lambda r: get_premium(r, options_data.loc[i]), axis=1)
        net_premium = (straddle.position * straddle.premium).sum()

        # Update mark_to_market dataframe
        mark_to_market = add_to_mtm(mark_to_market, straddle, i)

        # Exit the trade if any of the exit condition is met
        if options_data.loc[i, 'signal_iv'] == 0:
            exit_type = 'Expiry'
            exit_flag = True

        elif net_premium < sl:
            exit_type = 'SL'
            exit_flag = True

        elif net_premium > tp:
            exit_type = 'TP'
            exit_flag = True

        if exit_flag:

            # Append the trades dataframe
            trades['exit_date'] = i
            trades['exit_type'] = exit_type
            trades['exit_price'] = straddle.premium

            # Calculate net premium at exit
            net_premium = round(
                (straddle.position * straddle.premium).sum(), 1)

            # Calculate net premium on entry
            entry_net_premium = (trades.position * trades.entry_price).sum()

            # Calculate pnl for the trade
            trade_pnl = round(net_premium - entry_net_premium, 1)
            trades['PnL'] = trade_pnl

            # Add the trade logs to round trip details
            round_trips_details = pd.concat([round_trips_details, trades])

            # Calculate cumulative pnl
            cum_pnl += trade_pnl
            cum_pnl = round(cum_pnl, 2)

            # Print trade details
            print(
                f"Trade No: {trade_num} | Exit Type: {exit_type} | Date: {i} | Premium: {net_premium} | PnL: {trade_pnl} | Cum PnL: {cum_pnl}")

            # Update current position to 0
            current_position = 0

            # Set exit flag to false
            exit_flag = False

------------------------------
Trade No: 1 | Entry | Date: 2020-06-12 00:00:00 | Premium: -191.4 | Pnl: 0 | Cum PnL: 0
Trade No: 1 | Exit Type: TP | Date: 2020-06-25 00:00:00 | Premium: -75.0 | PnL: 116.4 | Cum PnL: 116.4
------------------------------
Trade No: 2 | Entry | Date: 2020-06-26 00:00:00 | Premium: -68.6 | Pnl: 0 | Cum PnL: 116.4
Trade No: 2 | Exit Type: SL | Date: 2020-07-01 00:00:00 | Premium: -169.6 | PnL: -101.0 | Cum PnL: 15.4
------------------------------
Trade No: 3 | Entry | Date: 2020-07-02 00:00:00 | Premium: -169.2 | Pnl: 0 | Cum PnL: 15.4
Trade No: 3 | Exit Type: TP | Date: 2020-07-27 00:00:00 | Premium: -55.6 | PnL: 113.6 | Cum PnL: 129.0
------------------------------
Trade No: 4 | Entry | Date: 2020-07-28 00:00:00 | Premium: -54.2 | Pnl: 0 | Cum PnL: 129.0
Trade No: 4 | Exit Type: SL | Date: 2020-08-03 00:00:00 | Premium: -138.8 | PnL: -84.6 | Cum PnL: 44.4
------------------------------
Trade No: 5 | Entry | Date: 2020-09-03 00:00:00 | Premium: -183.9 | Pnl

In [8]:
# Round trip details
round_trips_details.head()

,Option Type,Strike Price,position,entry_price,entry_date,exit_date,exit_type,exit_price,PnL
0,CE,3040.0,-1,77.80,2020-06-12,2020-06-25,TP,33.80,116.4
1,PE,3040.0,-1,113.55,2020-06-12,2020-06-25,TP,41.22,116.4
0,CE,3010.0,-1,34.08,2020-06-26,2020-07-01,SL,85.85,-101.0
1,PE,3010.0,-1,34.47,2020-06-26,2020-07-01,SL,83.70,-101.0
0,CE,3130.0,-1,87.23,2020-07-02,2020-07-27,TP,26.90,113.6


In [9]:
# MTM details
mark_to_market.head()

,Option Type,Strike Price,position,premium,Date
0,CE,3040.0,-1,77.80,2020-06-12
1,PE,3040.0,-1,113.55,2020-06-12
0,CE,3040.0,-1,75.91,2020-06-15
1,PE,3040.0,-1,81.30,2020-06-15
0,CE,3040.0,-1,68.92,2020-06-16


In [10]:
# Trade Level Analytics
analytics = trade_level_analytics(round_trips_details)
analytics

,Strategy
Total PnL,4856.550000
total_trades,29.000000
Number of Winners,18.000000
Number of Losers,11.000000
Win (%),62.068966
Loss (%),37.931034
per_trade_PnL_winners,625.013889
per_trade_PnL_losers,581.245455
Profit Factor,1.759584


## Conclusion

In this notebook, we backtested a short straddle strategy which takes trade based on the forecasted IV values and exits the same based on stop-loss, take-profit, or on expiry. You can either try changing the IV threshold value or you can deploy a different strategy, like a long straddle. You can also use minute-wise or bid-ask data for more accurate performance.

**Note: Short straddle is a risky strategy. To avoid the risk you can consider hedged strategies such as the butterfly strategy.**
<br><br>  